# Reusing `importlib`, `nbconvert`, and `nbformat` to import notebooks

In [1]:
    from importlib.machinery import SourceFileLoader, FileFinder
    from importlib.util import decode_source
    import sys, inspect
    from nbconvert.exporters.python import PythonExporter
    from nbformat.v4 import reads

## Adding new loader details to an existing FileFinder

In [12]:
    def find_finder():
        for i, hook in enumerate(sys.path_hooks):
            try:
                if issubclass(inspect.getclosurevars(hook).nonlocals['cls'], FileFinder
                             ):return i, hook
            except: ...

## The Notebook Source File Loader

In [13]:
    class NotebookLoader(SourceFileLoader):
        def source_to_code(self, data, path=None):
            return compile(PythonExporter().from_notebook_node(
                reads(decode_source(data))
            )[0], path, 'exec')

* Using the existing machinery for the SourceFileLoader means that we will created cached version of the import notebook source code.
* Moreover, reload will work.

In [14]:
    Ø = __name__ == '__main__'

In [15]:
    if Ø:
        sys.path_hooks[1] = FileFinder.path_hook(
             (NotebookLoader, ('.ipynb',)),*(
                 object for object in inspect.getclosurevars(find_finder()[1]).nonlocals['loader_details']
                 if not any(map('.ipynb'.__eq__, object[1]))))

* Clear the cache or enter a debugging nightmaare.

In [6]:
    if Ø: sys.path_importer_cache.clear()

* Check the brakes on some wonky settings.

In [7]:
    if Ø:
        ip = get_ipython()
        ip.ast_transformers = []

In [9]:
    if Ø:
        import foobar
        assert isinstance(foobar.__loader__, NotebookLoader), "Some other thing is loaded."

Show it creates a cached copy.